<a href="https://colab.research.google.com/github/paresh-nayyar/CSCI-544_NLP/blob/master/NLP_Project_Joy_LSTM%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
import string

In [2]:
url = 'https://raw.githubusercontent.com/paresh-nayyar/CSCI-544_NLP/master/data/code_mix_sentences_emotions_tagged.csv?token=GHSAT0AAAAAABTJRLBZ5MDZHWSYQSJMB6KOYSWNJVA'
df = pd.read_csv(url)

print(df.head(5))

  category                                         Statements
0    happy                     esh bar ka summer the best tha
1    angry                               sala chutiya hai log
2  neutral         I just came back yesterday from California
3    happy         family ke sat califonia gumane me maza aya
4    happy  ghar ke sab logo ko thankyou mera sat rhane ka...


In [3]:
df.columns

Index(['category', 'Statements'], dtype='object')

In [4]:
df.category.value_counts()

happy       142
sad          79
angry        70
neutral      70
fear         30
surprise      7
Name: category, dtype: int64

In [5]:
len(df)

398

In [6]:
def cleanSentence(sentence):
  sentence = sentence.lower()
  sentence = sentence.rstrip().replace("\n"," ").replace("\r"," ")
  sentence = re.sub(' +', ' ', sentence)
  sentence = sentence.translate(str.maketrans('', '', string.punctuation))

  return sentence

df["Statements"]= df["Statements"].apply(lambda x : cleanSentence(x))
df.head()

,category,Statements
0,happy,esh bar ka summer the best tha
1,angry,sala chutiya hai log
2,neutral,i just came back yesterday from california
3,happy,family ke sat califonia gumane me maza aya
4,happy,ghar ke sab logo ko thankyou mera sat rhane ka...


In [7]:
def tokenize(sentence):
  tokens = sentence.split(" ")
  tokens = [x for x in tokens if len(x)>0]
  return tokens

df["tokens"]= df["Statements"].apply(lambda x : tokenize(x))
df.head()

,category,Statements,tokens
0,happy,esh bar ka summer the best tha,"[esh, bar, ka, summer, the, best, tha]"
1,angry,sala chutiya hai log,"[sala, chutiya, hai, log]"
2,neutral,i just came back yesterday from california,"[i, just, came, back, yesterday, from, califor..."
3,happy,family ke sat califonia gumane me maza aya,"[family, ke, sat, califonia, gumane, me, maza,..."
4,happy,ghar ke sab logo ko thankyou mera sat rhane ka...,"[ghar, ke, sab, logo, ko, thankyou, mera, sat,..."


In [8]:
def getVocab(df):
  vocab = set()
  for tokens in df["tokens"]:
    tokens = set(tokens)
    vocab = vocab.union(tokens)
  return list(vocab)

vocab = getVocab(df)
len(vocab)

1990

In [9]:
", ".join(vocab)

'pardesh, badiya, express, gana, hor, ya, rhi, rahen, there, need, border, tea, masala, kabada, hath, agar, dat, sahi, pas, 3, jaan, sundar, aour, bakwass, nok, love, situatn, desires, har, gaal, chahte, bhaut, cangan, totaly, laundiya, enjoy, talented, mumy, confusing, sharam, lg, california, kadmon, daalke, listening, meh, pi, tarfa, score, feel, yr, wai, boreeee, pahali, hello, armaan, indira, modi, prblms, bhagega, tumhe, hotey, hmari, mut, evry, monk, world, country, bad, topper, where, dept, lammmmbbbbbaaaaaa, my, maang, kill, haha, dijie, kch, aswome, de, fantastic, apas, mushaq, oonpe, fielder, jaisi, nigaah, army, ple, yehi, bete, jaaye, form, yeh, luck, watching, q, sochna, pdega, gya, abhinetri, bhaav, lol, dono, diz, short, banoge, if, kro, ka, kardiya, they, breaking, mili, dekhengey, dur, hppy, achhe, opening, sucks, admsn, id, chutiya, halat, aunty, miss, jhnd, sorry, kahne, pak, bhi, pani, bigad, moments, phele, jagah, iitb, kaaran, bs, india, cut, kinda, dekhna, uneki,

In [10]:
!pip install fasttext
import fasttext
import numpy as np
import tensorflow as tf

     |████████████████████████████████| 68 kB 4.7 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3148851 sha256=914b3627e6a5d615df68b10f82a6b6043960ceec79f1ec4e78628ab7c7bde4d6
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [11]:
# Maybe there is a better way, some questions are best left unanswered..
df['Statements'].to_csv(r'./pandas.txt', header=None, index=None, sep=' ', mode='a')
em_model = fasttext.train_unsupervised(r'./pandas.txt')

In [12]:
em_model.get_nearest_neighbors('meaning')
len(em_model.get_word_vector('hais'))

100

In [13]:
# Build the embedding matrix
vector_size = 100
vocab_size = len(vocab)
em_matrix = np.zeros((vocab_size, vector_size))
for i in range(len(vocab)):
  em_matrix[i] = em_model.get_word_vector(vocab[i])

In [20]:
# Where the magic needs a sexy assistant
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, vector_size, weights=[em_matrix]))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 100)         199000    
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 128)        84480     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, None, 64)          8256      
                                                                 
 dropout_2 (Dropout)         (None, None, 64)          0         
                                                                 
 dense_5 (Dense)             (None, None, 1)           65        
                                                                 
Total params: 291,801
Trainable params: 291,801
Non-trainable params: 0
________________________________________________

State of the art model here ladies and gentlemen! So good that the loss is negative

In [21]:
label_encoding = {
  'happy': 0,
  'sad':  1,
  'angry':  2,
  'neutral':  3,
  'fear': 4,
  'surprise': 5,
}

max_len = max(df['tokens'].str.len())
train_data = []
train_label = df["category"].apply(lambda x: label_encoding.get(x)).to_numpy()

def prepare_train_data(tokens):
  for token in tokens:
    token_vector = em_model.get_word_vector(token)
    np.append(token_vector, [0] * (max_len - len(token_vector)))

  train_data.append(token_vector)

df["tokens"].apply(lambda x: prepare_train_data(x))


train_data = np.array(train_data)
# tf.keras.preprocessing.sequence.pad_sequences(train_data, maxlen=max_len)

print(train_data.shape)

## TODO: Added padding to train_data before feeding into the model but missing something

history = model.fit(train_data, train_label.reshape(-1, 1), batch_size=512, epochs=32, validation_split=0.3, callbacks=[callback])

(398, 100)
Epoch 1/32
1/1 [==============================] - 6s 6s/step - loss: 0.6921 - accuracy: 0.2272 - val_loss: 0.6710 - val_accuracy: 0.1417
Epoch 2/32
1/1 [==============================] - 1s 873ms/step - loss: 0.6830 - accuracy: 0.2230 - val_loss: 0.6416 - val_accuracy: 0.1417
Epoch 3/32
1/1 [==============================] - 1s 871ms/step - loss: 0.6697 - accuracy: 0.2230 - val_loss: 0.6071 - val_accuracy: 0.1417
Epoch 4/32
1/1 [==============================] - 1s 874ms/step - loss: 0.6538 - accuracy: 0.2230 - val_loss: 0.5658 - val_accuracy: 0.1417
Epoch 5/32
1/1 [==============================] - 1s 903ms/step - loss: 0.6354 - accuracy: 0.2230 - val_loss: 0.5142 - val_accuracy: 0.1417
Epoch 6/32
1/1 [==============================] - 1s 942ms/step - loss: 0.6120 - accuracy: 0.2230 - val_loss: 0.4489 - val_accuracy: 0.1417
Epoch 7/32
1/1 [==============================] - 1s 864ms/step - loss: 0.5833 - accuracy: 0.2230 - val_loss: 0.3655 - val_accuracy: 0.1417
Epoch 8/32
1